# Demo: Visualize Streaming Data with InfluxDB & Prospective

## Setup

In [1]:
# add parent directory to python search path
import sys
sys.path.append('..')       # must run this to import package from parent directory

from time import sleep
from datetime import datetime, timedelta
from generators.smart_grid.new_york_smart_grid import NewYorkSmartGridStreamGenerator
from writers.influxdb_writer import InfluxdbWriter
from writers.file_writer import FileWriter
from utils.logger import logger

# ============================================
# DEMO CONFIGURATION PARAMETERS
# change these parameters to customize the demo
# ============================================
START_TIME = datetime.now() - timedelta(minutes=0)              # start 5 minutes ago
START_TIME = datetime.now()
END_TIME = datetime.now() + timedelta(minutes=1)               # end 30 minutes from now
INFLUXDB_URL = 'http://localhost:8086'
INFLUXDB_TOKEN = 'sudo-banana-404'
INFLUXDB_ORG = 'perspective'
INFLUXDB_BUCKET = 'smart_grid'


def run():
    # log demo start message
    logger.info('-' * 50)
    logger.info('Demo: Starting New York Smart Grid demo...')

    # --------------------------------------------
    # DATA GENERATORS & WRITERS

    smart_grid = NewYorkSmartGridStreamGenerator(
        interval=0.250,         # refresh every 0.25 seconds
        nrows=1000,             # number of rows to generate in each batch
        num_stations=32,        # number of stations
        start_time=START_TIME,  # start time
        end_time=END_TIME,      # end time
        loopback=False,         # do not loop back. end the demo after end_time is reached
        )

    # Define the InfluxDB writer
    influx_writer = InfluxdbWriter(
        url=INFLUXDB_URL,
        token=INFLUXDB_TOKEN,
        org=INFLUXDB_ORG,
        bucket=INFLUXDB_BUCKET,
        measurement='new_york_smart_grid',
        timestamp_col='timestamp',
        tag_cols=['station_name', 'latitude', 'longitude'],
        field_cols=['energy_consumption', 'current', 'voltage']
        )
    
    # setup complete
    logger.info('Demo: Setup complete. Starting data generation...')

    # --------------------------------------------
    # MAIN STREAMING LOOP
    smart_grid.add_subscriber(influx_writer.write)
    smart_grid.start()
    # wait for the demo loop to finish or until the user interrupts
    try:
        while smart_grid.is_running():
            sleep(1)
    except KeyboardInterrupt:
        logger.info('Demo: Execution interrupted by the user. Ending demo...')
        smart_grid.stop()
    finally:
        # Ensure the InfluxDB writer is closed properly
        influx_writer.close()
        del influx_writer
        del smart_grid
        logger.info('Demo: Ended New York Smart Grid demo.')
    


run()

2024-12-11 22:08:18 - INFO - --------------------------------------------------------------------------------
2024-12-11 22:08:18 - INFO - App: started
2024-12-11 22:08:18 - INFO - --------------------------------------------------
2024-12-11 22:08:18 - INFO - Demo: Starting New York Smart Grid demo...
2024-12-11 22:08:18 - DEBUG - NewYorkSmartGridStreamGenerator: status=initialized, stations=32, nrows=1000, loopback=False
2024-12-11 22:08:18 - INFO - InfluxdbWriter: Initialized with url=http://localhost:8086, org=perspective, bucket=smart_grid, measurement=new_york_smart_grid
2024-12-11 22:08:18 - INFO - Demo: Setup complete. Starting data generation...
2024-12-11 22:08:18 - DEBUG - Stream-Generator::StartGenThread
2024-12-11 22:08:18 - INFO - Stream-Generator: start_time=2024-12-11 22:08:18.530290, end_time=2024-12-11 22:09:18.530308, interval=0.25, nrows=1000
2024-12-11 22:08:18 - DEBUG - Stream-Generator::DataGenTime: 37.796 ms
2024-12-11 22:08:18 - DEBUG - InfluxdbWriter: Wrote 32